In [9]:
import numpy as np
import time
from sklearn.metrics import accuracy_score
import scipy.optimize as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import math
from tabulate import tabulate
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean, std
import warnings
warnings.filterwarnings("ignore")

## Data Edit & Clean

In [10]:
# load data
df =pd.read_csv('https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv')

## Data Splitting

Split the dataset into training, validation and testing set (0.6, 0.2, 0.2)

In [11]:
# We focus on the sensitive attribute: race
# Caucasian: 1; African-American: 0
is_race = [i=='African-American' or i=='Caucasian' for i in df.race]
df = df.loc[is_race]

In [12]:
# If the charge date of a defendants Compas scored crime was not within 30 days from when the person was arrested, 
# we assume that because of data quality reasons, that we do not have the right offense.
is_date = [i<=30 and i>=-30 for i in df.days_b_screening_arrest]
df = df.loc[is_date]

In [13]:
# The recidivist flag -- is_recid -- is encoded as -1 if we could not find a compas case at all.
# Remove rows with is_recid=-1
df = df.loc[df.is_recid != -1]

In [14]:
# Those with a c_charge_degree of 'O' -- will not result in Jail time are removed
df = df.loc[df.c_charge_degree != "O"]

In [15]:
# dropna and select required columns
col = df.isna().sum().sort_values()[-14:].index.tolist()
df = df.drop(columns = col)
df = df.dropna()
df = df.loc[df.score_text != 'N/A',['sex','race','decile_score','two_year_recid']]

In [16]:
# Turn 'sex' and 'race' into dummy variables
df['sex'].replace(['Male','Female'],[1,0], inplace = True)
df['race'].replace(['Caucasian','African-American'],[1,0], inplace = True)

In [17]:
# Data splitting
# training : validation : testing = 5:1:1
data_train, data_test = train_test_split(df, test_size=0.14, random_state=15)
data_train, data_val= train_test_split(data_train, test_size=0.16, random_state=15)

# Learning Fair Representations

$d(x_n, v_k) = ||x_n − v_k||_2$
<br>
$d(x_n, v_k, α) = \sum\limits _{i=1}^{D} α_i(x_{ni} - v_{ki})^2$


In [18]:
# distance - d(x_n, v_k, alpha)
def d(X, v, alpha):
    N = X.shape[0]
    D = X.shape[1]
    K = len(v)
    res = np.zeros((N, K))
    for n in range(N):
        for k in range(K):
            for d in range(D):
                res[n, k] += alpha[d]*(X.iloc[n][d] - v[k, d])**2
    
    return res

$M_{n,k}=P(Z=k|x_n)=exp(-d(x,v_k))/\sum\limits_{j=1}^{k}exp(-d(x,v_j))$

In [19]:
def M_nk(dist, k):
    N, K = dist.shape
    expo_res = np.exp(-1 * dist)
    deno = np.sum(expo_res, axis=1).reshape(-1, 1)
    M_nk = expo_res / deno
    return M_nk

$M_k = \frac{1}{|X_0|} \sum_{n \in X_0} M_{nk}$

In [20]:
def M_k(X, M_nk, k):
    N, K = M_nk.shape
    M_k_values = np.sum(M_nk, axis=0) / N
    return M_k_values

$
\hat{x}_n = \sum^K_{k=1}M_{nk}v_k
$


$
L_x = \sum_{n=1}^N (x_n - \hat{x}_n)^2
$

In [21]:
#  the reconstruction of x_n  and L_x
def xnhat(X, M_nk, v):
    N,K = M_nk.shape
    D = X.shape[1]
    x_n_hat = np.zeros((N, D))
    L_x = 0
    for n in range(N):
        for d in range(D):
            for k in range(K):
                x_n_hat[n, d] += M_nk[n, k]*v[k, d]
        # calculate L_x        
        L_x += (X.iloc[n][d] - x_n_hat[n, d])**2   
    return x_n_hat, L_x

$\hat{y_n}=\sum\limits_{k = 1}^{K}M_{n,k}w_k$, we constrain the $w_k$ values to be between 0 and 1.

$L_y = \sum_{n=1}^N -y_n log \hat{y}_n - (1-y_n)log(1- \hat{y}_n) $

In [22]:
def ynhat(M_nk, w, y):
    N, K = M_nk.shape
    y_hat = np.dot(M_nk, w)
    log_loss_terms = -y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat)
    L_y = log_loss_terms.sum()
    return y_hat, L_y

We are going to minimize the Object function $L$

$L = A_z L_Z + A_x L_x + A_y L_y$ 


In [23]:
def L(params, sensitive_data, nonsensitive_data, sensitive_labels, nonsensitive_labels, K, A_z, A_x, A_y):
    sensitive_samples, sensitive_features = sensitive_data.shape
    nonsensitive_samples, nonsensitive_features = nonsensitive_data.shape

    alpha_sensitive = params[:sensitive_features]
    alpha_nonsensitive = params[sensitive_features : 2 * sensitive_features]
    w = params[2 * sensitive_features : (2 * sensitive_features) + K]
    v = np.matrix(params[(2 * sensitive_features) + K:]).reshape((K, sensitive_features))

    dist_sensitive = d(sensitive_data, v, alpha_sensitive)
    dist_nonsensitive = d(nonsensitive_data, v, alpha_nonsensitive)

    M_nk_sensitive = M_nk(dist_sensitive, K)
    M_nk_nonsensitive = M_nk(dist_nonsensitive, K)

    M_k_sensitive = M_k(sensitive_data, M_nk_sensitive, K)
    M_k_nonsensitive = M_k(nonsensitive_data, M_nk_nonsensitive, K)

    L_z = sum(abs(M_k_sensitive[k] - M_k_nonsensitive[k]) for k in range(K))

    x_hat_sensitive, L_x_sensitive = xnhat(sensitive_data, M_nk_sensitive, v)
    x_hat_nonsensitive, L_x_nonsensitive = xnhat(nonsensitive_data, M_nk_nonsensitive, v)
    L_x = L_x_sensitive + L_x_nonsensitive

    y_hat_sensitive, L_y_sensitive = ynhat(M_nk_sensitive, w, sensitive_labels)
    y_hat_nonsensitive, L_y_nonsensitive = ynhat(M_nk_nonsensitive, w, nonsensitive_labels)
    L_y = L_y_sensitive + L_y_nonsensitive

    loss = A_z * L_z + A_x * L_x + A_y * L_y

    return loss

In [24]:
def cal_pred(params, D, K, sen_dt, nsen_dt, sen_label, nsen_label):
    # Extract and reshape parameters
    alphas_sen = params[:D]
    alphas_nsen = params[D : 2 * D]
    weights = params[2 * D : (2 * D) + K]
    v_matrix = np.matrix(params[(2 * D) + K:]).reshape((K, D))
    
    # Compute distance matrices for sen and nsen data
    dist_sen = d(sen_dt, v_matrix, alphas_sen)
    dist_nsen = d(nsen_dt, v_matrix, alphas_nsen) 
    
    # Compute M_nk matrices for sen and nsen data
    M_nk_sen = M_nk(dist_sen, K)
    M_nk_nsen = M_nk(dist_nsen, K)
    
    # Compute y_n_hat matrices and likelihoods for sen and nsen data
    y_hat_sen, likelihood_sen = ynhat(M_nk_sen, weights, sen_label)
    y_hat_nsen, likelihood_nsen = ynhat(M_nk_nsen, weights, nsen_label)
    
    return y_hat_sen, y_hat_nsen

In [25]:
# calculates the overall accuracy, 
# accuracy for African-American group
# accuracy for Caucasian group
# and calibration of the model

def cal_calibr(y_pred_sensitive, y_pred_nonsensitive, y_sensitive_labels, y_nonsensitive_labels):

    # Convert the predictions using the threshold function
    converted_y_pred_sensitive = [1 if pred >= 0.5 else 0 for pred in y_pred_sensitive] 
    converted_y_pred_nonsensitive = [1 if pred >= 0.5 else 0 for pred in y_pred_nonsensitive] 

    # Calculate the accuracy for sensitive and nonsensitive groups
    accuracy_sensitive = accuracy_score(y_sensitive_labels, converted_y_pred_sensitive)
    accuracy_nonsensitive = accuracy_score(y_nonsensitive_labels, converted_y_pred_nonsensitive)

    # Combine the labels and predictions for calculating the overall accuracy
    combined_labels = y_sensitive_labels.append(y_nonsensitive_labels)
    combined_predictions = np.concatenate((converted_y_pred_sensitive, converted_y_pred_nonsensitive), axis=0)
    
    # Calculate the overall accuracy
    overall_accuracy = accuracy_score(combined_labels, combined_predictions)

    # Calculate the calibration of the model
    calibration = abs(accuracy_sensitive - accuracy_nonsensitive)

    print("The overall accuracy is: ", overall_accuracy)
    print("The accuracy for African-American is: ", accuracy_sensitive)
    print("The accuracy for Caucasian is: ", accuracy_nonsensitive)
    print("The calibration of the model is: ", calibration)

In [26]:
def LFR(training_data, val_data, y_name, sensitive_variable_name, K, A_z, A_x, A_y):
    # Split the training dataset into sensitive and non-sensitive groups
    sensitive_training = training_data[training_data[sensitive_variable_name] == 0]
    non_sensitive_training = training_data[training_data[sensitive_variable_name] == 1]

    # Split the validation dataset
    sensitive_validation = val_data[val_data[sensitive_variable_name] == 0]
    non_sensitive_validation = val_data[val_data[sensitive_variable_name] == 1]

    # Remove sensitive variable
    for dataset in [sensitive_training, sensitive_validation, non_sensitive_training, non_sensitive_validation]:
        dataset.drop(columns=[sensitive_variable_name], inplace=True)

    # Assign target variable (y) and remove it from the datasets
    y_sensitive_training = sensitive_training.pop(y_name)
    y_sensitive_validation = sensitive_validation.pop(y_name)
    y_non_sensitive_training = non_sensitive_training.pop(y_name)
    y_non_sensitive_validation = non_sensitive_validation.pop(y_name)

    # Generate random values for alpha and w
    alpha_sensitive = np.random.dirichlet(np.ones(sensitive_training.shape[1]), size=1).flatten()
    alpha_non_sensitive = np.random.dirichlet(np.ones(non_sensitive_training.shape[1]), size=1).flatten()
    w = np.random.dirichlet(np.ones(K), size=1).flatten()
    v = np.random.random((K, sensitive_training.shape[1]))

    initial_parameters = np.concatenate([alpha_sensitive, alpha_non_sensitive, w, v.flatten()])

    # Define parameter boundaries
    bounds = [(0, 1)] * (sensitive_training.shape[1] + non_sensitive_training.shape[1] + K) + [(None, None)] * (K * sensitive_training.shape[1])

    # Minimize the metric
    optimal_params, min_L, _ = optim.fmin_l_bfgs_b(L, x0=initial_parameters, epsilon=1e-5,
                                                   args=(sensitive_training, non_sensitive_training, y_sensitive_training,
                                                         y_non_sensitive_training, K, A_z, A_x, A_y),
                                                   bounds=bounds, approx_grad=True,
                                                   maxfun=150000, maxiter=150000)

    # Predict y_n_hat for the training dataset
    y_hat_sensitive_train, y_hat_non_sensitive_train = cal_pred(optimal_params, sensitive_training.shape[1], K, sensitive_training,
                                                                 non_sensitive_training, y_sensitive_training, y_non_sensitive_training)

    print("Training accuracy:")
    cal_calibr(y_hat_sensitive_train, y_hat_non_sensitive_train, y_sensitive_training, y_non_sensitive_training)

    # Predict y_n_hat for the validation dataset
    y_hat_sensitive_val, y_hat_non_sensitive_val = cal_pred(optimal_params, sensitive_validation.shape[1], K, sensitive_validation,
                                                             non_sensitive_validation, y_sensitive_validation, y_non_sensitive_validation)

    print("Validation accuracy:")
    cal_calibr(y_hat_sensitive_val, y_hat_non_sensitive_val, y_sensitive_validation, y_non_sensitive_validation)

    return optimal_params

In [27]:
start = time.time()
para_test = LFR(data_train, data_val, 'two_year_recid', 'race', 10, 0.3, 0.3, 0.4)
end = time.time() 
print( f"The training time: {end-start}")

Training accuracy:
The overall accuracy is:  0.53125
The accuracy for African-American is:  0.4842740198190435
The accuracy for Caucasian is:  0.6045729657027572
The calibration of the model is:  0.1202989458837137
Validation accuracy:
The overall accuracy is:  0.5165289256198347
The accuracy for African-American is:  0.4671201814058957
The accuracy for Caucasian is:  0.5929824561403508
The calibration of the model is:  0.12586227473445516
The training time: 663.7078614234924


In [28]:
# Test the LFR model
sensitive_test = data_test[data_test['race']==0]
nsensitive_test = data_test[data_test['race']==1]

sensitive_test=sensitive_test.drop(columns=['race'])
nsensitive_test = nsensitive_test.drop(columns=['race'])

y_sensitive_test = sensitive_test['two_year_recid']
sensitive_test = sensitive_test.drop(columns=['two_year_recid'])

y_nsensitive_test = nsensitive_test['two_year_recid']
nsensitive_test = nsensitive_test.drop(columns=['two_year_recid'])

start = time.time()
y_hat_sen_test, y_hat_nsen_test = cal_pred(para_test, sensitive_test.shape[1], 10, sensitive_test, 
             nsensitive_test, y_sensitive_test, y_nsensitive_test)
end = time.time() 
print( f"runtime of testing LFR model: {end-start}")
cal_calibr(y_hat_sen_test, y_hat_nsen_test, y_sensitive_test, y_nsensitive_test)

runtime of testing LFR model: 0.44855690002441406
The overall accuracy is:  0.530446549391069
The accuracy for African-American is:  0.44282238442822386
The accuracy for Caucasian is:  0.6402439024390244
The calibration of the model is:  0.19742151801080055
